# Pre-requisite libraries

In [48]:
!pip install pymongo
!pip install requests
!pip install tqdm
!pip install numpy

You should consider upgrading via the '/Users/cfrancois/campus/.venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/cfrancois/campus/.venv/bin/python3 -m pip install --upgrade pip' command.
     |████████████████████████████████| 78 kB 696 kB/s eta 0:00:01
You should consider upgrading via the '/Users/cfrancois/campus/.venv/bin/python3 -m pip install --upgrade pip' command.


# Extract info

**WARNING: The following code supposes that your install one OpenFoodFact MongoDB instance locally.**

In [14]:
from pymongo import MongoClient
import requests, shutil
import re
from tqdm import tqdm
import json

In [2]:
def parse(string, prefix=prefix):
    code = string['code']
    if len(code) > 8:
        tmp = re.split(r'(...)(...)(...)(.*)$', code)[1:-1]
    else:
        tmp = code
    newprefix = prefix + '/'.join(tmp) + '/'
    names = list(string['images'].keys())
    urls = [f'{newprefix}{s}.400.jpg' for s in names]
    return urls

def save_image(image_url, parent_name):
    r = requests.get(image_url, stream = True)
    # Check if the image was retrieved successfully
    if r.status_code == 200:
        # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
        r.raw.decode_content = True
        # Open a local file with wb ( write binary ) permission.
        filename = parent_name + '_' + image_url.split('/')[-1]
        with open(filename,'wb') as f:
            shutil.copyfileobj(r.raw, f)

In [ ]:
client = MongoClient('localhost', 27017)
db = client['off']
collection = db['products']
prefix = 'https://images.openfoodfacts.org/images/products/'

In [5]:
projection = {
    'id':1,
    'code':1,
    'images':1,
    'abbreviated_product_name':1,
    'product_name':1,
    }
query = {
    'countries':{'$nin':['France', 'en:fr']},
    'images':{"$exists":True, '$ne':[]}
    }
cursor = collection.find(query, projection=projection, limit=100000)

In [6]:
ids = [i for i in cursor]
image_url_dict= {i['code']:parse(i) for i in ids}

In [10]:
# build dict with code as key and product_name and/or
# abbreviated_product_name when they exist
id_name_dict = {
    i['code']:{
    'product_name':i.get('product_name', 'NA'),
    'abbreviated_product_name':i.get('abbreviated_product_name', 'NA')
    } for i in ids
}
# save the dict into .json file.
with open('label_of_products.json', 'w') as file:
     file.write(json.dumps(id_name_dict))

# Download images one by one (long, maybe could be optimized)

In [27]:
from pathlib import Path
DOWNLOADED_IMAGE_LIST = Path('images_downloaded.txt')


if DOWNLOADED_IMAGE_LIST.exists():
    with open('images_downloaded.txt', 'r') as f:
        downloaded = f.readlines()
    # set is faster than list for existence
    downloaded = set([i.strip() for i in downloaded])
else:
    downloaded = set()

In [28]:
for k, urls in tqdm(image_url_dict.items()):
    for image_url in urls:
        if k in downloaded:
            pass
        else:
            save_image(image_url, 'images/'+k)
        downloaded.add(k)

 14%|█▍        | 13939/100000 [00:17<02:35, 553.81it/s]  

In [26]:
with open('images_downloaded.txt', 'w') as f:
    for line in downloaded:
        f.write(f"{line}\n")